In [1]:
from ai import ai2s, ai14s
from marubatsu import Marubatsu

movelist = [(0, 0), (1, 0), (2, 0), (0, 1), (2, 1), (1, 1), (1, 2), (2, 2), (0, 2)]

mb = Marubatsu()
for x, y in movelist:
    print(mb)
    print("ai2s")
    %timeit ai2s(mb)
    print("ai14s")
    %timeit ai14s(mb)
    print()
    mb.move(x, y)

Turn o
...
...
...

ai2s
362 µs ± 15.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
ai14s
530 µs ± 8.74 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn x
O..
...
...

ai2s
328 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
ai14s
488 µs ± 2.71 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn o
oX.
...
...

ai2s
306 µs ± 14.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
ai14s
454 µs ± 12.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn x
oxO
...
...

ai2s
277 µs ± 11.7 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
ai14s
401 µs ± 11.9 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn o
oxo
X..
...

ai2s
236 µs ± 1.93 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
ai14s
333 µs ± 1.52 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn x
oxo
x.O
...

ai2s
213 µs ± 28.6 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
ai14s
288 µs

In [2]:
from ai import ai_match

ai_match(ai=[ai2s, ai14s])

ai2s VS ai14s


100%|██████████| 10000/10000 [01:12<00:00, 138.75it/s]

count     win    lose    draw
o           0    8818    1182
x           0    9914      86
total       0   18732    1268

ratio     win    lose    draw
o        0.0%   88.2%   11.8%
x        0.0%   99.1%    0.9%
total    0.0%   93.7%    6.3%



[('count',
  [{'win': 0, 'lose': 8818, 'draw': 1182},
   {'win': 0, 'lose': 9914, 'draw': 86},
   {'win': 0, 'lose': 18732, 'draw': 1268}],
  '7d'),
 ('ratio',
  [{'win': 0.0, 'lose': 0.8818, 'draw': 0.1182},
   {'win': 0.0, 'lose': 0.9914, 'draw': 0.0086},
   {'win': 0.0, 'lose': 0.9366, 'draw': 0.0634}],
  '7.1%')]

In [3]:
from ai import dprint
from functools import wraps
from random import choice

def ai_by_score(eval_func):
    @wraps(eval_func)
    def wrapper(mb_orig, debug=False, *args, rand=True, 
                analyze=False, calc_score=False, minimax=False, **kwargs):
        if calc_score:
            score = eval_func(mb_orig, debug, *args, **kwargs)
            if minimax and mb_orig.turn == Marubatsu.CIRCLE:
                score *= -1
            return score

        dprint(debug, "Start ai_by_score")
        dprint(debug, mb_orig)
        legal_moves = mb_orig.calc_legal_moves()
        dprint(debug, "legal_moves", legal_moves)
        best_score = float("-inf")
        best_moves = []
        if analyze:
            score_by_move = {}
        for move in legal_moves:
            dprint(debug, "=" * 20)
            dprint(debug, "move", move)
            x, y = move
            mb_orig.move(x, y)
            dprint(debug, mb_orig)
            score = eval_func(mb_orig, debug, *args, **kwargs)
            mb_orig.unmove()
            dprint(debug, "score", score, "best score", best_score)
            if analyze:
                score_by_move[move] = score
            
            if best_score < score:
                best_score = score
                best_moves = [move]
                dprint(debug, "UPDATE")
                dprint(debug, "  best score", best_score)
                dprint(debug, "  best moves", best_moves)
            elif best_score == score:
                best_moves.append(move)
                dprint(debug, "APPEND")
                dprint(debug, "  best moves", best_moves)

        dprint(debug, "=" * 20)
        dprint(debug, "Finished")
        dprint(debug, "best score", best_score)
        dprint(debug, "best moves", best_moves)
        if analyze:
            return {
                "candidate": best_moves,
                "score_by_move": score_by_move,
            }
        elif rand:   
            return choice(best_moves)
        else:
            return best_moves[0]
        
    return wrapper

In [4]:
from marubatsu import Markpat

@ai_by_score
def ai2s(mb, debug=False):
    return 0

@ai_by_score
def ai14s(mb, debug=False, score_victory=300, score_sure_victory=200, \
          score_defeat=-100, score_special=100, score_201=2, \
          score_102=0.5, score_012=-1):
    # 評価値の合計を計算する変数を 0 で初期化する
    score = 0        

    # 自分が勝利している場合
    if mb.status == mb.last_turn:
        return score_victory

    markpats = mb.count_markpats()
    if debug:
        pprint(markpats)
    # 相手が勝利できる場合は評価値を加算する
    if markpats[Markpat(last_turn=0, turn=2, empty=1)] > 0:
        score = score_defeat * markpats[Markpat(last_turn=0, turn=2, empty=1)]
    # 次の自分の手番で自分が必ず勝利できる場合
    elif markpats[Markpat(last_turn=2, turn=0, empty=1)] >= 2:
        return score_sure_victory
    
    # 斜め方向に 〇×〇 が並び、いずれかの辺の 1 つのマスのみに × が配置されている場合
    if mb.board[1][1] == Marubatsu.CROSS and \
        (mb.board[0][0] == mb.board[2][2] == Marubatsu.CIRCLE or \
        mb.board[2][0] == mb.board[0][2] == Marubatsu.CIRCLE) and \
        (mb.board[1][0] == Marubatsu.CROSS or \
        mb.board[0][1] == Marubatsu.CROSS or \
        mb.board[2][1] == Marubatsu.CROSS or \
        mb.board[1][2] == Marubatsu.CROSS) and \
        mb.move_count == 4:
        return score_special    

    # 次の自分の手番で自分が勝利できる場合は評価値に score_201 を加算する
    if markpats[Markpat(last_turn=2, turn=0, empty=1)] == 1:
        score += score_201
    # 「自 1 敵 0 空 2」1 つあたり score_102 だけ、評価値を加算する
    score += markpats[Markpat(last_turn=1, turn=0, empty=2)] * score_102
    # 「自 0 敵 1 空 2」1 つあたり score_201 だけ、評価値を減算する
    score += markpats[Markpat(last_turn=0, turn=1, empty=2)] * score_012
    
    # 計算した評価値を返す
    return score

In [5]:
mb = Marubatsu()
for x, y in movelist:
    print(mb)
    print("ai2s")
    %timeit ai2s(mb)
    print("ai14s")
    %timeit ai14s(mb)
    print()
    mb.move(x, y)

Turn o
...
...
...

ai2s
179 µs ± 2.42 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
ai14s
335 µs ± 3.06 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn x
O..
...
...

ai2s
160 µs ± 2.55 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
ai14s
304 µs ± 10.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn o
oX.
...
...

ai2s
139 µs ± 1.66 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
ai14s
283 µs ± 25.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn x
oxO
...
...

ai2s
120 µs ± 1.58 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
ai14s
245 µs ± 19.8 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn o
oxo
X..
...

ai2s
104 µs ± 1.99 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
ai14s
189 µs ± 4.71 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Turn x
oxo
x.O
...

ai2s
81.4 µs ± 1.35 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
ai14s

In [6]:
ai_match(ai=[ai2s, ai14s])

ai2s VS ai14s


100%|██████████| 10000/10000 [00:36<00:00, 273.53it/s]

count     win    lose    draw
o           0    8800    1200
x           0    9897     103
total       0   18697    1303

ratio     win    lose    draw
o        0.0%   88.0%   12.0%
x        0.0%   99.0%    1.0%
total    0.0%   93.5%    6.5%



[('count',
  [{'win': 0, 'lose': 8800, 'draw': 1200},
   {'win': 0, 'lose': 9897, 'draw': 103},
   {'win': 0, 'lose': 18697, 'draw': 1303}],
  '7d'),
 ('ratio',
  [{'win': 0.0, 'lose': 0.88, 'draw': 0.12},
   {'win': 0.0, 'lose': 0.9897, 'draw': 0.0103},
   {'win': 0.0, 'lose': 0.93485, 'draw': 0.06515}],
  '7.1%')]

In [7]:
from random import choice

mb = Marubatsu()
for x, y in movelist:
    print(mb)
    print("合法手の計算処理")
    %timeit mb.calc_legal_moves()
    print("ai2s での評価値の計算処理")
    %timeit ai2s(mb, calc_score=True)
    print("ai14s での評価値の計算処理")
    %timeit ai14s(mb, calc_score=True)
    print("choice の計算処理")
    legal_moves = mb.calc_legal_moves()
    %timeit choice(legal_moves)
    print()
    mb.move(x, y)


Turn o
...
...
...

合法手の計算処理
1.67 µs ± 20.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
ai2s での評価値の計算処理
274 ns ± 11.9 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
ai14s での評価値の計算処理
17.2 µs ± 312 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
choice の計算処理
294 ns ± 13.7 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)

Turn x
O..
...
...

合法手の計算処理
1.66 µs ± 16.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
ai2s での評価値の計算処理
276 ns ± 11.6 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
ai14s での評価値の計算処理
16.9 µs ± 239 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
choice の計算処理
292 ns ± 1.17 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)

Turn o
oX.
...
...

合法手の計算処理
1.65 µs ± 24.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
ai2s での評価値の計算処理
269 ns ± 1.4 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)
ai14s での評価値の計算処理
16.6 µs ± 145 ns per

In [18]:
%%timeit

best_score = 0
score = 1
move = (1, 1)
best_moves = []
maxnode = True
if (maxnode and best_score < score) or (not maxnode and best_score > score):
    best_score = score
    best_moves = [move]
elif best_score == score:
    best_moves.append(move)

76.7 ns ± 1.21 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [19]:
%%timeit

best_score = 2
score = 1
move = (1, 1)
best_moves = []
maxnode = False
if (maxnode and best_score < score) or (not maxnode and best_score > score):
    best_score = score
    best_moves = [move]
elif best_score == score:
    best_moves.append(move)

83.2 ns ± 2.53 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [20]:
%%timeit

best_score = 1
score = 1
move = (1, 1)
best_moves = []
maxnode = True
if (maxnode and best_score < score) or (not maxnode and best_score > score):
    best_score = score
    best_moves = [move]
elif best_score == score:
    best_moves.append(move)

107 ns ± 0.937 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [25]:
%%timeit

best_score = 2
score = 1
move = (1, 1)
best_moves = []
maxnode = True
if (maxnode and best_score < score) or (not maxnode and best_score > score):
    best_score = score
    best_moves = [move]
elif best_score == score:
    best_moves.append(move)

66.4 ns ± 2.92 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [26]:
%%timeit

best_score = 0
score = 1
move = (1, 1)
best_moves = []
maxnode = False
if (maxnode and best_score < score) or (not maxnode and best_score > score):
    best_score = score
    best_moves = [move]
elif best_score == score:
    best_moves.append(move)

64.7 ns ± 0.495 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [22]:
%%timeit

best_score = 2
score = 1
move = (1, 1)
best_moves = []
maxnode = True

34.1 ns ± 0.288 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
from ai import ai_abs_dls

eval_params = {"minimax": True}

In [13]:
mb = Marubatsu()
for x, y in movelist:
    print(mb)
    %timeit ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=9)
    print()
    mb.move(x, y)

Turn o
...
...
...

33.7 ms ± 187 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Turn x
O..
...
...

25.7 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Turn o
oX.
...
...

22.7 ms ± 2.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Turn x
oxO
...
...

6.67 ms ± 133 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Turn o
oxo
X..
...

3.25 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Turn x
oxo
x.O
...

1.44 ms ± 17 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn o
oxo
xXo
...

436 µs ± 1.76 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn x
oxo
xxo
.O.

223 µs ± 19.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn o
oxo
xxo
.oX

82.5 µs ± 2.31 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)



In [14]:
from copy import deepcopy

def ai_by_mmscore(eval_func):
    @wraps(eval_func)
    def wrapper(mb_orig, debug=False, *args, rand=True, share_tt=True,
                analyze=False, calc_score=False, **kwargs):       
        if calc_score:
            score, count = eval_func(mb_orig, debug, *args, **kwargs)
            return score
        
        starttime = perf_counter()
        dprint(debug, "Start ai_by_mmscore")
        dprint(debug, mb_orig)
        legal_moves = mb_orig.calc_legal_moves()
        dprint(debug, "legal_moves", legal_moves)
        maxnode = mb_orig.turn == Marubatsu.CIRCLE
        best_score = float("-inf") if maxnode else float("inf")
        best_moves = []
        tt = {} if share_tt else None
        totalcount = 0
        if analyze:
            score_by_move = {}
        for move in legal_moves:
            dprint(debug, "=" * 20)
            dprint(debug, "move", move)
            x, y = move
            mb_orig.move(x, y)
            dprint(debug, mb_orig)
            score, count = eval_func(mb_orig, debug, tt=tt, *args, **kwargs)
            mb_orig.unmove()
            totalcount += count
            dprint(debug, "score", score, "best score", best_score)
            if analyze:
                score_by_move[move] = score
            
            if (maxnode and best_score < score) or (not maxnode and best_score > score):
                best_score = score
                best_moves = [move]
                dprint(debug, "UPDATE")
                dprint(debug, "  best score", best_score)
                dprint(debug, "  best moves", best_moves)
            elif best_score == score:
                best_moves.append(move)
                dprint(debug, "APPEND")
                dprint(debug, "  best moves", best_moves)

        dprint(debug, "=" * 20)
        dprint(debug, "Finished")
        dprint(debug, "best score", best_score)
        dprint(debug, "best moves", best_moves)
        bestmove = choice(best_moves) if rand else best_moves[0]
        if analyze:
            if share_tt:
                PV = []
                mb = deepcopy(mb_orig)
                while mb.status == Marubatsu.PLAYING:
                    PV.append(bestmove)
                    x, y = bestmove
                    if mb.board[x][y] != Marubatsu.EMPTY:
                        print("そのマスには着手済みです")
                        break
                    mb.move(x, y)
                    boardtxt = mb.board_to_str()
                    if boardtxt in tt:
                        _, _, bestmove = tt[boardtxt]
                    else:
                        break                
            else:
                PV = bestmove
            return {
                "candidate": best_moves,
                "score_by_move": score_by_move,
                "tt": tt,
                "time": perf_counter() - starttime,
                "bestmove": PV[0],
                "score": best_score,
                "count": totalcount,
                "PV": PV,
            }
        else:
            return bestmove
        
    return wrapper

In [15]:
from time import perf_counter

@ai_by_mmscore
def ai_abs_dls(mb:Marubatsu, debug:bool=False, timelimit_pc:float|None=None, maxdepth:int=1,
               eval_func=None, eval_params:dict={}, use_tt:bool=False,
               tt:dict|None=None, tt_for_mo:dict|None=None) -> tuple[float, int]:
   
    count = 0
    def ab_search(mborig, depth, tt, alpha=float("-inf"), beta=float("inf")):
        nonlocal count
        if timelimit_pc is not None and perf_counter() >= timelimit_pc:
            raise RuntimeError("time out")
        
        count += 1
        if mborig.status != Marubatsu.PLAYING or depth == maxdepth:
            return eval_func(mborig, calc_score=True, **eval_params)
        
        if use_tt:
            boardtxt = mborig.board_to_str()
            if boardtxt in tt:
                lower_bound, upper_bound, _ = tt[boardtxt]
                if lower_bound == upper_bound:
                    return lower_bound
                elif upper_bound <= alpha:
                    return upper_bound
                elif beta <= lower_bound:
                    return lower_bound
                else:
                    alpha = max(alpha, lower_bound)
                    beta = min(beta, upper_bound)
            else:
                lower_bound = min_score
                upper_bound = max_score
        
        alphaorig = alpha
        betaorig = beta

        legal_moves = mborig.calc_legal_moves()
        if tt_for_mo is not None:
            if not use_tt:            
                boardtxt = mborig.board_to_str()
            if boardtxt in tt_for_mo:
                _, _, bestmove = tt_for_mo[boardtxt]
                index = legal_moves.index(bestmove)
                legal_moves[0], legal_moves[index] = legal_moves[index], legal_moves[0]        
        if mborig.turn == Marubatsu.CIRCLE:
            score = float("-inf")
            for x, y in legal_moves:
                mborig.move(x, y)
                abscore = ab_search(mborig, depth + 1, tt, alpha, beta)
                mborig.unmove()
                if abscore > score:
                    bestmove = (x, y)
                score = max(score, abscore)
                if score >= beta:
                    break
                alpha = max(alpha, score)
        else:
            score = float("inf")
            for x, y in legal_moves:
                mborig.move(x, y)
                abscore = ab_search(mborig, depth + 1, tt, alpha, beta)
                mborig.unmove()
                if abscore < score:
                    bestmove = (x, y)
                score = min(score, abscore)
                if score <= alpha:
                    break
                beta = min(beta, score)   
            
        from util import calc_same_boardtexts

        if use_tt:
            boardtxtlist = calc_same_boardtexts(mborig, bestmove)
            if score <= alphaorig:
                upper_bound = score
            elif score < betaorig:
                lower_bound = score
                upper_bound = score
            else:
                lower_bound = score
            for boardtxt, move in boardtxtlist.items():
                tt[boardtxt] = (lower_bound, upper_bound, move)
        return score
                
    min_score = float("-inf")
    max_score = float("inf")
    
    if tt is None:
        tt = {}
    score = ab_search(mb, depth=0, tt=tt, alpha=min_score, beta=max_score)
    dprint(debug, "count =", count)
    return score, count

In [16]:
from util import Check_solved

Check_solved.is_strongly_solved(ai_abs_dls, params={"eval_func": ai14s, "eval_params": eval_params, "use_tt": True, "maxdepth": 8})

100%|██████████| 431/431 [00:01<00:00, 304.20it/s]

431/431 100.00%


(True, [])

In [17]:
mb = Marubatsu()
for x, y in movelist:
    print(mb)
    %timeit ai_abs_dls(mb, eval_func=ai14s, eval_params=eval_params, use_tt=True, maxdepth=9)
    print()
    mb.move(x, y)

Turn o
...
...
...

32.7 ms ± 561 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)

Turn x
O..
...
...

25.5 ms ± 2.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Turn o
oX.
...
...

22.7 ms ± 3.34 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Turn x
oxO
...
...

6.21 ms ± 103 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Turn o
oxo
X..
...

2.95 ms ± 17.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)

Turn x
oxo
x.O
...

1.28 ms ± 25.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn o
oxo
xXo
...

338 µs ± 14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)

Turn x
oxo
xxo
.O.

145 µs ± 2.44 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

Turn o
oxo
xxo
.oX

44.2 µs ± 936 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)

